# Tweets analyzer.

#### This Notebook analyzes tweets to from individuals to a company. The please note:
##### 1. The main goal of this notebook is to analyze questions and reviews left by users for company's therefore to get more relevant data I have excluded tweets with attachments and only included tweets which are not replies to tweets or retweets.

#### 2. The data has been scrapped from twitter directly using octoparse 8. This data is from the periods between 01/01/2021 to 31/12/2021.

In [141]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install spacy-langdetect
!pip install pyspellchecker

import pandas as pd
import re
import plotly.express as px
import numpy as np
import datetime
import spacy

^C


In [2]:
df = pd.read_csv('data/kplc_twitter_2021.csv')

In [3]:
def convertCountToInt(count):
    if(isinstance(count,str) and count[-1].lower() in ['k','m']):
        multiplier = 1000  if count[-1].lower() == 'k' else 1000000
        count = float(count[:len(count)-1]) * multiplier
    return int(count)

def cleanTweet1(tweet):
    pattern = r'\n'
    return re.sub(pattern,' ',tweet)

In [4]:
#converting comments, reshare and likes to int
df.fillna(0,inplace=True)

intColumns = ['comments','reshare','likes']

for column in intColumns:
    df[column] = df[column].apply(convertCountToInt);
    


In [5]:
#we'll remove next line tags to cleanup the data
df['tweet_cleaned'] = df['tweet'].apply(cleanTweet1)

#checking whether there are any null columns
df.isnull().sum()

Name             0
handle           0
Time             0
tweet            0
comments         0
reshare          0
likes            0
tweet_cleaned    0
dtype: int64

In [6]:
#we'll remove the name column since we don't need it for the analysis
df.drop(['Name'],axis=1,inplace = True)

df.sample(10, random_state=10)

,handle,Time,tweet,comments,reshare,likes,tweet_cleaned
1636,@seanC_1O,2021-05-04T15:44:36.000Z,Tangu Friday KPLC wamekataa kutengeneza stima ...,1,1,2,Tangu Friday KPLC wamekataa kutengeneza stima ...
1126,@amanialvinn,2021-07-28T09:50:08.000Z,#kplccustomercare \n@KenyaPower_Care\n \n@Keny...,1,0,0,#kplccustomercare @KenyaPower_Care @KenyaPo...
1583,@lydssham,2021-05-12T15:05:01.000Z,"Dear Customer,Your complaint for prepaid meter...",0,0,1,"Dear Customer,Your complaint for prepaid meter..."
1911,@kakayoung01,2021-04-08T15:00:10.000Z,Hello \n@KenyaPower_Care\n \n@KenyaPower\n fo...,0,0,0,Hello @KenyaPower_Care @KenyaPower for th...
404,@ansibuor,2021-11-16T12:53:36.000Z,"Now that we are talking about fraudsters, \n@K...",1,3,10,"Now that we are talking about fraudsters, @Ke..."
1305,@injilipaul,2021-06-27T08:39:04.000Z,No power! Since Saturday 26th june mid morning...,1,0,0,No power! Since Saturday 26th june mid morning...
822,@hingittah,2021-09-15T18:42:42.000Z,"....with fuel prices up, will our hydroelectri...",0,0,0,"....with fuel prices up, will our hydroelectri..."
707,@reggaeholic_,2021-10-06T14:28:05.000Z,"Sijawahi ona \n@KenyaPower_Care\n , \n@KenyaPo...",0,0,0,"Sijawahi ona @KenyaPower_Care , @KenyaPower..."
2189,@MosesNtabo,2021-03-10T07:28:56.000Z,I hope this >> \n@sos_kibe_\n not \n@KenyaPow...,1,0,0,I hope this >> @sos_kibe_ not @KenyaPower ...
1214,@KituiOnline,2021-07-11T17:26:22.000Z,Kitui: There are trees interfering with powerl...,0,0,2,Kitui: There are trees interfering with powerl...


In [7]:
# we need to format the date in a more analsis friendly way
df['Month'] = pd.DatetimeIndex(df['Time']).month
df['Day'] = pd.DatetimeIndex(df['Time']).day
df['Time'] = pd.DatetimeIndex(df['Time']).time

df.sample(10, random_state=7)

,handle,Time,tweet,comments,reshare,likes,tweet_cleaned,Month,Day
2640,@2FingersKenya,16:38:06,Hi \n@KenyaPower\n \n@KenyaPower_Care\n power ...,1,0,0,Hi @KenyaPower @KenyaPower_Care power outa...,1,13
1673,@johnmark254,18:08:30,#pawasawa \n@KenyaPower\n \n@KenyaPower_Care...,1,0,0,#pawasawa @KenyaPower @KenyaPower_Care ...,4,30
732,@Birchok_H,18:40:08,Dear \n@KenyaPower_Care\n kindly assist with t...,1,0,0,Dear @KenyaPower_Care kindly assist with tok...,10,3
267,@AllanKamau12,11:08:40,Hello \n@KenyaPower_Care\n \n@KenyaPower\n we...,1,0,0,Hello @KenyaPower_Care @KenyaPower we exp...,11,29
450,@Mercymunyingi,07:38:51,#kplccustomercare \n@kenyapower\n \n@KenyaPowe...,1,1,0,#kplccustomercare @kenyapower @KenyaPower_C...,11,9
2586,@Mca_Trickyy,10:30:05,Dear \n@KenyaPower\n \n@KenyaPower_Care\n same...,1,0,0,Dear @KenyaPower @KenyaPower_Care same 'bl...,1,24
412,@ItsGasheri,14:16:01,Oyaaaaaaa \n@KenyaPower_Care\n \n@KenyaPower\n...,2,0,0,Oyaaaaaaa @KenyaPower_Care @KenyaPower No ...,11,15
2677,@MichKatami,10:35:18,"""Mwangaza wamlika, nchi ya songa Mbele...""\n\n...",1,0,0,"""Mwangaza wamlika, nchi ya songa Mbele..."" ""P...",1,10
219,@stevemusilih,10:11:15,Good afternoon \n@KenyaPower\n \n@KenyaPower_C...,1,0,0,Good afternoon @KenyaPower @KenyaPower_Care...,12,3
2180,@ihaji_marilu,16:49:03,It's so unfortunate for a busy street like Kee...,0,1,0,It's so unfortunate for a busy street like Kee...,3,11


In [33]:
from spacy.language import Language
from spacy_langdetect import LanguageDetector
def get_lang_detector(nlp, name):
    return LanguageDetector()

LanguageClassifier = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
LanguageClassifier.add_pipe('language_detector', last=True)

In [45]:
#since we have tweets in both swahili and english, we will first categorize them into the respective language.
df['language'] = df['tweet'].apply(lambda tweet: 'Swahili' if LanguageClassifier(tweet)._.language['language']== 'sw' else 'English')

In [50]:
df.sample(10, random_state=7)

,handle,Time,tweet,comments,reshare,likes,tweet_cleaned,Month,Day,language
2640,@2FingersKenya,16:38:06,Hi \n@KenyaPower\n \n@KenyaPower_Care\n power ...,1,0,0,Hi @KenyaPower @KenyaPower_Care power outa...,1,13,English
1673,@johnmark254,18:08:30,#pawasawa \n@KenyaPower\n \n@KenyaPower_Care...,1,0,0,#pawasawa @KenyaPower @KenyaPower_Care ...,4,30,English
732,@Birchok_H,18:40:08,Dear \n@KenyaPower_Care\n kindly assist with t...,1,0,0,Dear @KenyaPower_Care kindly assist with tok...,10,3,English
267,@AllanKamau12,11:08:40,Hello \n@KenyaPower_Care\n \n@KenyaPower\n we...,1,0,0,Hello @KenyaPower_Care @KenyaPower we exp...,11,29,English
450,@Mercymunyingi,07:38:51,#kplccustomercare \n@kenyapower\n \n@KenyaPowe...,1,1,0,#kplccustomercare @kenyapower @KenyaPower_C...,11,9,English
2586,@Mca_Trickyy,10:30:05,Dear \n@KenyaPower\n \n@KenyaPower_Care\n same...,1,0,0,Dear @KenyaPower @KenyaPower_Care same 'bl...,1,24,English
412,@ItsGasheri,14:16:01,Oyaaaaaaa \n@KenyaPower_Care\n \n@KenyaPower\n...,2,0,0,Oyaaaaaaa @KenyaPower_Care @KenyaPower No ...,11,15,English
2677,@MichKatami,10:35:18,"""Mwangaza wamlika, nchi ya songa Mbele...""\n\n...",1,0,0,"""Mwangaza wamlika, nchi ya songa Mbele..."" ""P...",1,10,Swahili
219,@stevemusilih,10:11:15,Good afternoon \n@KenyaPower\n \n@KenyaPower_C...,1,0,0,Good afternoon @KenyaPower @KenyaPower_Care...,12,3,English
2180,@ihaji_marilu,16:49:03,It's so unfortunate for a busy street like Kee...,0,1,0,It's so unfortunate for a busy street like Kee...,3,11,English


In [66]:
#Lets see the count of kiswahili vs English reviews
df['language'].value_counts()

fig = px.bar(df, 
             y=df['language'].value_counts().values, 
             x=df['language'].value_counts().index,
             title = "Counts of tweets by language",
            labels={"x":"Language","y":"Number of tweets"})
fig.show()


In [74]:
#Now let's get a feel of the average length of a tweet

df['tweetLength'] = df['tweet'].apply(lambda tweet:len(tweet))
fig = px.box(df,y='tweetLength',title="Average tweet length")
fig.show()

In [83]:
#lets get to see the distribution of the tweets over different months

fig = px.bar(df,
             y=df['Month'].value_counts().values, 
             x=df['Month'].value_counts().index,
             title = "Counts of tweets by Month",
            labels={"x":"Month","y":"Number of tweets"})
fig.show()

In [128]:
#Getting distribution on accordance to hour of the day

df['hour'] = df['Time'].apply(lambda time: time.hour)

fig = px.bar(df,
             y=df['hour'].value_counts().values, 
             x=df['hour'].value_counts().index,
             title = "Counts of tweets by Month",
            labels={"x":"Hour","y":"Number of tweets"},
            )
fig.show()

#### From the initial EDA we have the following info:
#### 1. The tweets are multilingual with the larger percentage being on English and the lesser is Kiswahili.
#### 2. The average length of a tweet is 148 characters which tells us that people don't write long tweets for this company
#### 3. The first quarter of the year saw relatively more tweets than the other quarters.
#### 4. Most people tweet in the morning hours (6am - 9am) and in the afternoon (2pm - 6pm)

#### We will proceed to analyze the English reviews since they are more as compared to the English once and we also have more resources for English compared to swahili

In [99]:
#We'll first of all get our subset of the English reviews

df_english = df[df['language'] == 'English'].copy()
df_english.drop(['handle','comments','reshare','likes','Time','language'], axis = 1, inplace= True)
df_english.sample(10, random_state=9)

,tweet,tweet_cleaned,Month,Day,tweetLength
1374,\n@KenyaPower_Care\n \n@KenyaPower\n,@KenyaPower_Care @KenyaPower,6,13,34
1334,Aye \n@KenyaPower\n \n@KenyaPower_Care\n tusi...,Aye @KenyaPower @KenyaPower_Care tusizoeane,6,22,48
1823,Working from home and then KPLC decided to do ...,Working from home and then KPLC decided to do ...,4,16,122
760,Gmorng \n\n@KenyaPower_Care\n\n \n\n@KenyaPowe...,Gmorng @KenyaPower_Care @KenyaPower ha...,9,28,188
917,No power in langata (jambo Estate) Ref No.7045...,No power in langata (jambo Estate) Ref No.7045...,8,28,138
827,"Hi, \n@KenyaPower\n \n@KenyaPower_Care\n power...","Hi, @KenyaPower @KenyaPower_Care power out...",9,15,239
1939,Hello \n@KenyaPower_Care\n and \n@KenyaPower\n...,"Hello @KenyaPower_Care and @KenyaPower , th...",4,6,185
697,Hi \n@KenyaPower_Care\n \n@KenyaPower\n what's...,Hi @KenyaPower_Care @KenyaPower what's tim...,10,7,97
2260,Greetings \n@KenyaPower_Care\n \n@KenyaPower\n...,Greetings @KenyaPower_Care @KenyaPower . Ki...,2,28,175
2082,Hey \n@KenyaPower_Care\n \n@KenyaPower\n we do...,Hey @KenyaPower_Care @KenyaPower we don't ...,3,25,108


In [134]:
#Lets no clean the data to so a more indepth analysis

#removing contractions, special charaters, spell checking

df_english['tweet_cleaned2'] = df_english['tweet_cleaned'].apply(lambda tweet:re.sub('[^\w\d\s]+','',tweet))

In [ ]:
from spellchecker import SpellChecker

def spellCheckSentence(tweet):
    words = []
    for word in tweet.split():
        words.append(spell.correction(word))
    return ' '.join(words)

spell = SpellChecker()
    
df_english['tweet_cleaned2'] = df_english['tweet_cleaned2'].apply(spellCheckSentence)

In [ ]:
#this is EDA

#get word count for tweets
#get word cloud

#remove contractions
#make lower case
#check spelling
#break to month on month and check most common word and polarity
#check word count